In [5]:
URL= 'https://raw.githubusercontent.com/qcitizen/sample-data/main/samples_daily_data.csv'

results = await fetch(URL)
text = await results.text()

filename = 'daily_data_clean_0618.csv'

with open(filename, 'w') as f:
    f.write(text)

data = pd.read_csv(filename, sep= ',')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   level_0          90 non-null     int64  
 1   participantnum   90 non-null     object 
 2   participantid    90 non-null     object 
 3   responseid       90 non-null     object 
 4   scoring          90 non-null     object 
 5   mdyest           90 non-null     bool   
 6   whatsubstyest    1 non-null      object 
 7   othersubstyest   90 non-null     bool   
 8   lsddosageyest    0 non-null      float64
 9   psildosageyest   1 non-null      object 
 10  otherdosageyest  90 non-null     bool   
 11  preparedprodyes  1 non-null      object 
 12  stack            1 non-null      object 
 13  stackedother     90 non-null     bool   
 14  mdtoday          90 non-null     bool   
 15  whatsubsttod     66 non-null     object 
 16  othersubsttod    90 non-null     bool   
 17  psildosagetod    6

In [6]:
columns_to_recode = ['connectedq','contemplativeq','creativeq','focusq']

for col in columns_to_recode:
    data[col] = data[col].replace([0,1,2,3,4], [1,2,3,4,5])

In [7]:
dependent_measures = ['connectedq','contemplativeq','creativeq','focusq','productiveq','wellbeingq','sleepq','dreamsq']

In [8]:
days_md_df = data[['Task_Days','mdtoday']]
days_md_df.to_csv('days_md_df.csv')

In [9]:
for i, row in data.iterrows():
    if row['mdtoday'] == True:
        cell = 'Yes'
    elif row['mdtoday'] == False:
        cell = 'No'
    else:
        cell = np.nan
    data.at[i, 'Microdosing'] = cell

In [10]:
md_col_df = pd.DataFrame(data["Microdosing"])

In [11]:
#To use for columns that are not encoded as bool
# answer_list = ["True","False","prefernotsay",'']
# convert_list=[1,-1,0,0]
# n2s = dict(zip(answer_list, convert_list))

# data['dreamsq'] = data['dreamsq'].replace(n2s)

In [12]:
data['dreamsq'] = data['dreamsq'].replace([True, False], [1, 0])

# Daily Data

## Visualizations with no missing data

In [13]:
values = []
for measure in dependent_measures:
    values.append({
        'Measure': measure,
        'Overall Mean': round(data[measure].mean(),2),
        'Microdosing Days Mean': round(data[data['mdtoday'] == True][measure].mean(),2),
        'Non-microdosing Days Mean': round(data[data['mdtoday'] == False][measure].mean(),2),
        'Median': data[measure].median(),
        'Missing Values (%)': ((data[measure].isna().sum())/len(data)) * 100,
        'Number of Responses': data[measure].notnull().sum(),
        'Maximal Value': data[measure].max()
    })
summary_df = pd.DataFrame(values)    
summary_df

,Measure,Overall Mean,Microdosing Days Mean,Non-microdosing Days Mean,Median,Missing Values (%),Number of Responses,Maximal Value
0,connectedq,4.67,4.61,4.83,5.0,0.0,90,5
1,contemplativeq,4.52,4.48,4.62,5.0,0.0,90,5
2,creativeq,4.13,4.03,4.42,4.0,0.0,90,5
3,focusq,4.02,3.98,4.12,4.0,0.0,90,5
4,productiveq,3.47,3.36,3.75,4.0,0.0,90,5
5,wellbeingq,3.58,3.55,3.67,4.0,0.0,90,5
6,sleepq,3.79,3.79,3.79,4.0,0.0,90,5
7,dreamsq,0.12,0.11,0.17,0.0,0.0,90,1


In [14]:
sub_df = data[data['Microdosing'] == 'Yes']

alt.Chart(sub_df).mark_line(point=True).encode(
    x='Task_Days',
    y='psildosagetod'
).properties(
    width=900,
    title= 'Microdosing Days and Doses'
)

alt.Chart(...)

In [15]:
print("Stacking substance and number of times reported:", data['stack'].value_counts())

Stacking substance and number of times reported: lionsmane, niacin    1
Name: stack, dtype: int64


In [16]:
a = alt.Chart(data).mark_circle(size=80).encode(
            x='Task_Days:N',
            y=alt.Y('dreamsq:Q', scale=alt.Scale(domain=(0,1)), axis=alt.Axis(values=[0,1], ticks=True, labels=True, grid=False)),
            color=alt.Color('Microdosing', scale=alt.Scale(domain=['No', 'Yes'], range=['orange', 'navy']))
        ).properties(
            width=900,
            height=100
        )
b = alt.Chart(data).mark_line(point=False, color='black',opacity=0.3).encode(
        x='Task_Days:N',
        y='dreamsq:Q'
    ).properties(
        width=900,
        height=100,
        title= ' Days with Vivid Dreams'
    )

a

alt.Chart(...)

In [17]:
dependent_measures2 = ['connectedq','contemplativeq','creativeq','focusq','productiveq','wellbeingq','sleepq']

In [18]:
charts = []
titles = ['Connectedness to Nature', 'Contemplation', 'Creativity', 'Focus', 'Productivity', 'Well-being', 'Sleep Quality']

for title, measure in zip(titles, dependent_measures2):
    row = []
    for col in range(1, 2):
        a = alt.Chart(data).mark_circle(size=80).encode(
            x='Task_Days:N',
            y=alt.Y(measure, scale=alt.Scale(domain=(1,5)), axis=alt.Axis(values=[1, 2, 3, 4, 5])),
            color=alt.Color('Microdosing', scale=alt.Scale(domain=['No', 'Yes'], range=['orange', 'navy']))
        ).properties(
            width=900,
            title= title
        )
        b = alt.Chart(data).mark_line(point=False, color='black',opacity=0.3).encode(
            x='Task_Days:N',
            y=measure
        ).properties(
            width=900,
        )
        row.append(a+b)
    charts.append(alt.hconcat(*row))
alt.vconcat(*charts)

alt.VConcatChart(...)

In [19]:
protocol = 1
for i, row in data.iterrows():
    if i == 0:
        protocol_end = False
    else:
        prev_row = data.iloc[i - 1]
        protocol_end = prev_row.mdtoday == False and row.mdtoday == True
        
    if protocol_end:
        protocol += 1
        
    data.at[i, "Protocol"] = f"Cluster {protocol}"

In [20]:
charts = []
for row_nr in range(1,5):
    row = []
    for col in range(1, 5):
        subplot_data = data[data.Protocol == f"Cluster {(row_nr - 1) * 4 + col}"]
        
        if subplot_data.shape[0] == 1:
            break
            
        a = alt.Chart(subplot_data).mark_circle(size=80).encode(
            x='Task_Days:N',
            y=alt.Y('creativeq:Q',scale=alt.Scale(domain=(1,5))),
            color=alt.Color('mdtoday',
                   scale=alt.Scale(
            domain=['false', 'true'],
            range=['orange', 'navy']))
        ).properties(
            width=200,
            height=200,
        )
        b = alt.Chart(subplot_data).mark_line(point=False, color='black',opacity=0.3).encode(
            x='Task_Days:N',
            y='creativeq:Q'
        ).properties(
            width=200,
            height=200,
        )
        row.append(a+b)
    charts.append(alt.hconcat(*row))
alt.vconcat(*charts)

alt.VConcatChart(...)

In [21]:
for window in range(2,8):
    for i, row in data.iterrows():
        if i+1 <= window:
            start_index = 0
        else:
            start_index = i + 1 - window

        rows_in_window = data[start_index:i+1]
        try:
            md_in_window = rows_in_window['Microdosing'].value_counts()['Yes']
        except KeyError:
            md_in_window = 0
        data.at[i, f'Microdosing_in_last_{window}_days (%)'] = round(md_in_window / window, 2)
    

In [22]:
for window in range(2,8):
    for i, row in data.iterrows():
        if i+1 <= window:
            start_index = 0
        else:
            start_index = i + 1 - window

        rows_in_window = data[start_index:i+1]
        try:
            md_in_window = rows_in_window['Microdosing'].value_counts()['Yes']
        except KeyError:
            md_in_window = 0
        data.at[i, f'Num_microdosing_days_in_last_{window}_days'] = md_in_window
    

In [23]:
alt.Chart(data).mark_circle(color='navy',size=200, opacity=0.35).encode(
    alt.X('Microdosing_in_last_7_days (%):Q', 
        axis=alt.Axis(format='%'),
    ),
    alt.Y('creativeq:Q', scale=alt.Scale(domain=(1,5)), axis=alt.Axis(values=[1, 2, 3, 4, 5])),
)

alt.Chart(...)

In [24]:
data.to_csv('N_of_1_data_0729.csv', index=False)

In [25]:
main = []
for window in range(2,8):   
    stripplot = alt.Chart(data, width=140).mark_circle(size=50, opacity=0.35).encode(
        x=alt.X(
            'jitter:Q',
            title=None,
            axis=alt.Axis(values=[0], ticks=True, grid=False, labels=False,),
            scale=alt.Scale(),
        ),
        y=alt.Y('creativeq:Q', scale=alt.Scale(domain=(1,5)), axis=alt.Axis(values=[1, 2, 3, 4, 5])),
        color=alt.Color(f'Microdosing_in_last_{window}_days (%):N', legend=None),
        column=alt.Column(
            f'Microdosing_in_last_{window}_days (%):N',
            header=alt.Header(
                labelAngle=0,
                titleOrient='top',
                labelOrient='bottom',
                labelAlign='center',
                labelPadding=323,
            ),
        ),
    ).transform_calculate(
        # Generate Gaussian jitter with a Box-Muller transform
        jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
    ).configure_facet(
        spacing=0
    ).configure_view(
        stroke=None
    )
    main.append(stripplot)
main[5]

alt.Chart(...)

In [26]:
main = []
for window in range(2,8):   
    stripplot = alt.Chart(data, width=140).mark_circle(size=50, opacity=0.35).encode(
        x=alt.X(
            'jitter:Q',
            title=None,
            axis=alt.Axis(values=[0], ticks=True, grid=False, labels=False),
            scale=alt.Scale(),
        ),
        y=alt.Y('creativeq:Q', scale=alt.Scale(domain=(1,5)), axis=alt.Axis(values=[1, 2, 3, 4, 5])),
        color=alt.Color(f'Num_microdosing_days_in_last_{window}_days:N', legend=None),
        column=alt.Column(
            f'Num_microdosing_days_in_last_{window}_days:N',
            header=alt.Header(
                labelAngle=0,
                titleOrient='top',
                labelOrient='bottom',
                labelAlign='right',
                labelPadding=326,
            ),
        ),
    ).transform_calculate(
        # Generate Gaussian jitter with a Box-Muller transform
        jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
    ).configure_facet(
        spacing=0
    ).configure_view(
        stroke=None
    )
    main.append(stripplot)
main[5]

alt.Chart(...)

In [29]:
# main = alt.hconcat(data=data).transform_calculate(
#         # Generate Gaussian jitter with a Box-Muller transform
#         jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
#     ).configure_facet(
#         spacing=0
#     ).configure_view(
#         stroke=None
#     )

# for window in range(2,8):   
#     stripplot = alt.Chart(data, width=140).mark_circle(size=50, opacity=0.3).encode(
#         x=alt.X(
#             'jitter:Q',
#             title=None,
#             axis=alt.Axis(values=[0], ticks=True, grid=False, labels=False),
#             scale=alt.Scale(),
#         ),
#         y=alt.Y('creativeq:Q'),
#         color=alt.Color(f'Microdosing_last_{window}_days:N', legend=None),
#         column=alt.Column(
#             f'Microdosing_last_{window}_days:N',
#             header=alt.Header(
#                 labelAngle=-90,
#                 titleOrient='top',
#                 labelOrient='bottom',
#                 labelAlign='right',
#                 labelPadding=3,
#             ),
#         ),
#     )
#     main |= stripplot
# main

In [27]:
one_month_data = data[data['Task_Days'] < 29]
month1_yes = one_month_data[one_month_data['Microdosing'] == 'Yes']
month1_no = one_month_data[one_month_data['Microdosing'] == 'No']
md_data = data[data['Microdosing'] == 'Yes']
no_md_data = data[data['Microdosing'] == 'No']

In [28]:
a = alt.Chart(one_month_data).mark_circle(size=60).encode(
    x='Task_Days:N',
    y=alt.Y('creativeq:Q', 
            scale=alt.Scale(domain=(1,5)), axis=alt.Axis(values=[1, 2, 3, 4, 5])),
            color=alt.Color('mdtoday:N', legend=alt.Legend(orient="left"))
)

b = alt.Chart(month1_yes, width=160).mark_circle(opacity=0.4, size=60).encode(
    x=alt.X(
            'jitter:Q',
            title='Microdosing Days',
            axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
            scale=alt.Scale(),
        ),
    y=alt.Y(
        'creativeq:Q',
        axis=alt.Axis(title=None, values=[1, 2, 3, 4, 5],ticks=False, grid=True,labels=True),
        scale=alt.Scale(domain=(1,5))
    ),
    color=alt.Color('mdtoday',
            scale=alt.Scale(
            domain=['false', 'true'],
            range=['orange', 'navy'])),
    order='mdtoday:N'
).transform_calculate(
        # Generate Gaussian jitter with a Box-Muller transform
        jitter='(sqrt(-2*log(random()))*cos(2*PI*random())*17)'
    )

c = alt.Chart(month1_no, width=160).mark_circle(size=60).encode(
    x=alt.X(
            'jitter:Q',
            title='Non-microdosing Days',
            axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
            scale=alt.Scale(),
        ),
    y=alt.Y(
        'creativeq:Q',
        axis=alt.Axis(title=None, values=[1, 2, 3, 4, 5],ticks=False, grid=True,labels=True),
        scale=alt.Scale(domain=(1,5))
    ),
    color=alt.Color('mdtoday',
            scale=alt.Scale(
            domain=['false', 'true'],
            range=['orange', 'navy'])),
    order='mdtoday:N'
).transform_calculate(
        # Generate Gaussian jitter with a Box-Muller transform
        jitter='(sqrt(-2*log(random()))*cos(2*PI*random())*1)'
    )

a|b|c

alt.HConcatChart(...)

In [29]:
def calc_offset(df, col):
    counts = df[col].value_counts()
    interval = 2
    for key, value in zip(counts.index, counts):
        if value % 2 == 0:
            dist_from_mid = interval / 2
        else:
            dist_from_mid = 0

        right_offsets = []
        for i in range(0, math.ceil(value / 2)):
            right_offsets.append(dist_from_mid + i * interval)
        left_offsets = np.array(right_offsets[::-1]) * -1
        offsets = np.concatenate((left_offsets, np.array(right_offsets)))
        offsets = np.unique(offsets)

        j = 0
        for i, row in df[df[col] == key].iterrows():
            df.at[i, f'Offset_{col}'] = offsets[j]
            j += 1
            

In [30]:
for measure in dependent_measures2:
    calc_offset(month1_yes, measure)
    calc_offset(month1_no, measure)
    calc_offset(md_data, measure)
    calc_offset(no_md_data, measure)

/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [31]:
# hard-coding samples (population) means based on the full df from deepnote

d = {'connectedq': [3.35, 3.35],
     'contemplativeq': [3.33, 3.33],
     'creativeq': [3.18, 3.18],
     'focusq': [3.28, 3.28],
     'productiveq': [3.27, 3.27],
     'wellbeingq': [3.36, 3.36],
     'sleepq': [3.49, 3.49],
     'Task_Days': [0, 28]}

sample_means = pd.DataFrame(d)

In [32]:
charts = []
titles = ['Connectedness to Nature', 'Contemplation', 'Creativity', 'Focus', 'Productivity', 'Well-being', 'Sleep Quality']

for title, measure in zip(titles, dependent_measures2):
    one_month_data[f"y_mean_{measure}"] = one_month_data[measure].mean()
    row = []
    for col in range(1, 2):
        a = alt.Chart(one_month_data).mark_circle(size=60).encode(
            x='Task_Days:N',
            y=alt.Y(measure, scale=alt.Scale(domain=(1,5)), axis=alt.Axis(values=[1, 2, 3, 4, 5])),
            color=alt.Color('mdtoday:N', legend=alt.Legend(orient="left"))
        ).properties(
            title=title)
        
        means_ppt = alt.Chart(one_month_data).mark_line(color='blue', point=False, strokeDash=[1,4], strokeWidth=2.9).encode(
        x='Task_Days:N',
        y=f"y_mean_{measure}:Q"
        )

        # population means
        means_sample = alt.Chart(sample_means).mark_line(color='grey', point=False).encode(
        x='Task_Days:N',
        y=alt.Y(measure, axis=alt.Axis(title=''))
        )
            
        b = alt.Chart(month1_yes, width=160).mark_circle(size=60).encode(
            x=alt.X(
                    f'Offset_{measure}:Q',
                    title='Microdosing Days',
                    axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                    scale=alt.Scale(),
                ),
            y=alt.Y(
                measure, scale=alt.Scale(domain=(1,5)),
                axis=alt.Axis(title=None, values=[1, 2, 3, 4, 5], ticks=False, grid=True,labels=False),
            ),
            color=alt.Color('mdtoday',
                    scale=alt.Scale(
                    domain=['false', 'true'],
                    range=['orange', 'navy'])),
            order='mdtoday:N'
        )

        c = alt.Chart(month1_no, width=160).mark_circle(size=60).encode(
            x=alt.X(
                    f'Offset_{measure}:Q',
                    title='Non-microdosing Days',
                    axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                    scale=alt.Scale(),
                ),
            y=alt.Y(
                measure, scale=alt.Scale(domain=(1,5)),
                axis=alt.Axis(title=None, values=[1, 2, 3, 4, 5], ticks=False, grid=True,labels=False),
            ),
            color=alt.Color('mdtoday',
                    scale=alt.Scale(
                    domain=['false', 'true'],
                    range=['orange', 'navy'])),
            order='mdtoday:N'
        )
        row.append(a+means_ppt+means_sample|b|c)
    charts.append(alt.hconcat(*row))
alt.vconcat(*charts)

<ipython-input-32-697d36138df4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_month_data[f"y_mean_{measure}"] = one_month_data[measure].mean()


alt.VConcatChart(...)

In [33]:
d = {'connectedq': [3.35, 3.35],
     'contemplativeq': [3.33, 3.33],
     'creativeq': [3.18, 3.18],
     'focusq': [3.28, 3.28],
     'productiveq': [3.27, 3.27],
     'wellbeingq': [3.36, 3.36],
     'sleepq': [3.49, 3.49],
     'Task_Days': [0, 90]}

sample_means2 = pd.DataFrame(d)

In [34]:
charts = []
titles = ['Connectedness to Nature', 'Contemplation', 'Creativity', 'Focus', 'Productivity', 'Well-being', 'Sleep Quality']

for title, measure in zip(titles, dependent_measures2):
    data[f"y_mean_{measure}"] = data[measure].mean()
    row = []
    for col in range(1, 2):
        a = alt.Chart(data, width=600).mark_circle(size=30).encode(
            x=alt.X('Task_Days:N', axis=alt.Axis(values=[0, 90], ticks=True, grid=False, labels=True)),
            y=alt.Y(measure, scale=alt.Scale(domain=(1,5)), axis=alt.Axis(values=[1, 2, 3, 4, 5])),
            color=alt.Color('mdtoday:N', legend=alt.Legend(orient="left"))
        ).properties(
            title=title)

        means_ppt = alt.Chart(data).mark_line(color='blue', point=False, strokeDash=[1,4], strokeWidth=2.9).encode(
        x='Task_Days:N',
        y=f"y_mean_{measure}:Q"
        )

        means_sample = alt.Chart(sample_means2).mark_line(color='grey', point=False).encode(
        x='Task_Days:N',
        y=alt.Y(measure, axis=alt.Axis(title=''))
        )
        
        b = alt.Chart(md_data, width=160).mark_circle(size=30).encode(
            x=alt.X(
                    f'Offset_{measure}:Q',
                    title='Microdosing Days',
                    axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                    scale=alt.Scale(),
                ),
            y=alt.Y(
                measure, scale=alt.Scale(domain=(1,5)),
                axis=alt.Axis(title=None, values=[1, 2, 3, 4, 5], ticks=False, grid=True,labels=False),
            ),
            color=alt.Color('mdtoday',
                    scale=alt.Scale(
                    domain=['false', 'true'],
                    range=['orange', 'navy'])),
            order='mdtoday:N'
        )

        c = alt.Chart(no_md_data, width=160).mark_circle(size=30).encode(
            x=alt.X(
                    f'Offset_{measure}:Q',
                    title='Non-microdosing Days',
                    axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                    scale=alt.Scale(),
                ),
            y=alt.Y(
                measure, scale=alt.Scale(domain=(1,5)),
                axis=alt.Axis(title=None, values=[1, 2, 3, 4, 5], ticks=False, grid=True,labels=False),
            ),
            color=alt.Color('mdtoday',
                    scale=alt.Scale(
                    domain=['false', 'true'],
                    range=['orange', 'navy'])),
            order='mdtoday:N'
        )
        row.append(a+means_ppt+means_sample|b|c)
    charts.append(alt.hconcat(*row))
alt.vconcat(*charts)

alt.VConcatChart(...)

## Visualizations with the missing data

In [35]:
df_nans = data[dependent_measures].mask(np.random.random(data[dependent_measures].shape) < .3)

df_nans = df_nans.join(md_col_df)

df_nans

,connectedq,contemplativeq,creativeq,focusq,productiveq,wellbeingq,sleepq,dreamsq,Microdosing
0,4.0,4.0,3.0,NaN,2.0,3.0,NaN,NaN,Yes
1,NaN,4.0,3.0,5.0,3.0,4.0,NaN,0.0,Yes
2,5.0,5.0,4.0,NaN,4.0,4.0,4.0,1.0,Yes
3,5.0,NaN,5.0,5.0,4.0,NaN,3.0,0.0,Yes
4,5.0,NaN,4.0,5.0,4.0,NaN,4.0,NaN,No
...,...,...,...,...,...,...,...,...,...
85,NaN,5.0,4.0,4.0,NaN,4.0,5.0,NaN,Yes
86,5.0,4.0,3.0,3.0,NaN,3.0,4.0,0.0,Yes
87,5.0,4.0,5.0,4.0,3.0,4.0,4.0,0.0,No
88,NaN,5.0,4.0,4.0,4.0,NaN,4.0,NaN,No


In [36]:
task_days = pd.DataFrame(data.Task_Days)

df_nans = df_nans.join(task_days)

In [37]:
df_nans.creativeq.isna().sum()

26

In [38]:
for i, row in df_nans.iterrows():
    if np.isnan(row['dreamsq']):
        cell = row['Task_Days']
    else:
        cell = np.nan
    df_nans.at[i, 'Task_Days_Missing'] = cell    
        
df_nans       

,connectedq,contemplativeq,creativeq,focusq,productiveq,wellbeingq,sleepq,dreamsq,Microdosing,Task_Days,Task_Days_Missing
0,4.0,4.0,3.0,NaN,2.0,3.0,NaN,NaN,Yes,0.0,0.0
1,NaN,4.0,3.0,5.0,3.0,4.0,NaN,0.0,Yes,1.0,NaN
2,5.0,5.0,4.0,NaN,4.0,4.0,4.0,1.0,Yes,2.0,NaN
3,5.0,NaN,5.0,5.0,4.0,NaN,3.0,0.0,Yes,3.0,NaN
4,5.0,NaN,4.0,5.0,4.0,NaN,4.0,NaN,No,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...
85,NaN,5.0,4.0,4.0,NaN,4.0,5.0,NaN,Yes,86.0,86.0
86,5.0,4.0,3.0,3.0,NaN,3.0,4.0,0.0,Yes,87.0,NaN
87,5.0,4.0,5.0,4.0,3.0,4.0,4.0,0.0,No,88.0,NaN
88,NaN,5.0,4.0,4.0,4.0,NaN,4.0,NaN,No,89.0,89.0


In [41]:
a = alt.Chart(df_nans).mark_circle(size=80, color='navy').encode(
            x='Task_Days:N',
            y=alt.Y('dreamsq:Q', scale=alt.Scale(domain=(0,1)), axis=alt.Axis(values=[0,1], ticks=True, labels=True, grid=False)),
            color=alt.Color('Microdosing:N', scale=alt.Scale(domain=['No', 'Yes'], range=['orange', 'navy']))
        ).properties(
            width=900,
            height=100
        )
b = alt.Chart(data).mark_line(point=False, color='black',opacity=0.3).encode(
        x='Task_Days:N',
        y='dreamsq:Q'
    ).properties(
        width=900,
        height=100,
        title= ' Days with Vivid Dreams'
    )

c = alt.Chart(df_nans).mark_rule(color='red').encode(
    x=alt.X('Task_Days_Missing:N'))

a+b+c

alt.LayerChart(...)

In [42]:
a = alt.Chart(data).mark_circle(size=80).encode(
            x='Task_Days:N',
            y=alt.Y('dreamsq:Q', scale=alt.Scale(domain=(0,1)), axis=alt.Axis(values=[0,1], ticks=True, labels=True, grid=False)),
            color=alt.Color('Microdosing:N', scale=alt.Scale(domain=['No', 'Yes'], range=['orange', 'navy']))
        ).properties(
            width=900,
            height=100
        )
b = alt.Chart(data).mark_line(point=False, color='black',opacity=0.3).encode(
        x='Task_Days:N',
        y='dreamsq:Q'
    ).properties(
        width=900,
        height=100,
        title= ' Days with Vivid Dreams'
    )

a+b

alt.LayerChart(...)

In [62]:
a = alt.Chart(df_nans).mark_circle(size=80, color='navy').encode(
            x='Task_Days:N',
            y=alt.Y('creativeq:Q', scale=alt.Scale(domain=(1,5)), axis=alt.Axis(values=[1,2,3,4,5], ticks=True, labels=True, grid=False)),
            color=alt.Color('Microdosing:N', scale=alt.Scale(domain=['No', 'Yes'], range=['orange', 'navy']))
        ).properties(
            width=1000,
            height=200
        )
b = alt.Chart(df_nans).mark_line(point=False, color='black',opacity=0.3).encode(
        x='Task_Days:N',
        y='creativeq:Q'
    ).properties(
        width=1000,
        height=200,
        title= 'Creativity ratings during the course of study'
    )

c = alt.Chart(df_nans).mark_rule(color='red').encode(
    x=alt.X('Task_Days_Missing:N'))

a+b+c

alt.LayerChart(...)

In [46]:
a = alt.Chart(df_nans).mark_circle(size=60).encode(
    x='Task_Days:N',
    y=alt.Y('creativeq:Q', 
            scale=alt.Scale(domain=(1,5)), axis=alt.Axis(values=[1, 2, 3, 4, 5])),
            color=alt.Color('mdtoday:N', legend=alt.Legend(orient="left"))
)

# b = alt.Chart(df_nans, width=160).mark_circle(opacity=0.4, size=60).encode(
#     x=alt.X(
#             'jitter:Q',
#             title='Microdosing Days',
#             axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
#             scale=alt.Scale(),
#             ),
#         y=alt.Y(
#             'creativeq:Q',
#             axis=alt.Axis(title=None, values=[1, 2, 3, 4, 5],ticks=False, grid=True,labels=True),
#             scale=alt.Scale(domain=(1,5))
#         ),
#         color=alt.Color('Microdosing:N',
#                 scale=alt.Scale(
#                 domain=['false', 'true'],
#                 range=['orange', 'navy'])),
#         order='mdtoday:N'
#     ).transform_calculate(
#             # Generate Gaussian jitter with a Box-Muller transform
#             jitter='(sqrt(-2*log(random()))*cos(2*PI*random())*17)'
#         )
        
c = alt.Chart(df_nans).mark_rule(color='red').encode(
    x=alt.X('Task_Days_Missing:N'))

a+c


# c = alt.Chart(month1_no, width=160).mark_circle(size=60).encode(
#     x=alt.X(
#             'jitter:Q',
#             title='Non-microdosing Days',
#             axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
#             scale=alt.Scale(),
#         ),
#     y=alt.Y(
#         'creativeq:Q',
#         axis=alt.Axis(title=None, values=[1, 2, 3, 4, 5],ticks=False, grid=True,labels=True),
#         scale=alt.Scale(domain=(1,5))
#     ),
#     color=alt.Color('mdtoday',
#             scale=alt.Scale(
#             domain=['false', 'true'],
#             range=['orange', 'navy'])),
#     order='mdtoday:N'
# ).transform_calculate(
#         # Generate Gaussian jitter with a Box-Muller transform
#         jitter='(sqrt(-2*log(random()))*cos(2*PI*random())*1)'
#     )

# a|b|c

alt.LayerChart(...)

In [28]:
highlight = alt.selection(type='single', on='mouseover', fields=['mdtoday'], nearest=True)

base = alt.Chart(data).encode(
    x='Task_Days:N',
    y='creativeq:Q',
    color=alt.Color('mdtoday:N')
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=900
)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3))
)

points + lines

alt.LayerChart(...)

In [2]:
!jupyter nbconvert --to pdf --TemplateExporter.exclude_input=True N_of_1_data_0729.ipynb

[NbConvertApp] Converting notebook N_of_1_data_0729.ipynb to pdf
[NbConvertApp] ERROR | Error while converting 'N_of_1_data_0729.ipynb'
Traceback (most recent call last):
  File "/Users/mags/miniforge3/lib/python3.9/site-packages/nbconvert/nbconvertapp.py", line 410, in export_single_notebook
    output, resources = self.exporter.from_filename(notebook_filename, resources=resources)
  File "/Users/mags/miniforge3/lib/python3.9/site-packages/nbconvert/exporters/exporter.py", line 179, in from_filename
    return self.from_file(f, resources=resources, **kw)
  File "/Users/mags/miniforge3/lib/python3.9/site-packages/nbconvert/exporters/exporter.py", line 197, in from_file
    return self.from_notebook_node(nbformat.read(file_stream, as_version=4), resources=resources, **kw)
  File "/Users/mags/miniforge3/lib/python3.9/site-packages/nbconvert/exporters/pdf.py", line 170, in from_notebook_node
    latex, resources = super(PDFExporter, self).from_notebook_node(
  File "/Users/mags/miniforge3

In [3]:
!pip3 install pandoc

In [4]:
import pandoc

In [7]:
!jupyter nbconvert --to pdf --TemplateExporter.exclude_input=True N_of_1_data_0729.ipynb

[NbConvertApp] Converting notebook N_of_1_data_0729.ipynb to pdf
[NbConvertApp] ERROR | Error while converting 'N_of_1_data_0729.ipynb'
Traceback (most recent call last):
  File "/Users/mags/miniforge3/lib/python3.9/site-packages/nbconvert/nbconvertapp.py", line 410, in export_single_notebook
    output, resources = self.exporter.from_filename(notebook_filename, resources=resources)
  File "/Users/mags/miniforge3/lib/python3.9/site-packages/nbconvert/exporters/exporter.py", line 179, in from_filename
    return self.from_file(f, resources=resources, **kw)
  File "/Users/mags/miniforge3/lib/python3.9/site-packages/nbconvert/exporters/exporter.py", line 197, in from_file
    return self.from_notebook_node(nbformat.read(file_stream, as_version=4), resources=resources, **kw)
  File "/Users/mags/miniforge3/lib/python3.9/site-packages/nbconvert/exporters/pdf.py", line 170, in from_notebook_node
    latex, resources = super(PDFExporter, self).from_notebook_node(
  File "/Users/mags/miniforge3